2D CNN on test_area dataset

In [ ]:
# Imports 
import numpy as np
import torch
import torch.nn as nn # nn models
import torch.optim as optim # optimization algorithms (incl. stocastic gradient descend)
import torch.nn.functional as F # activation functions (incl. ReLu)
from torch.utils.data import TensorDataset, DataLoader # dataset management
import torchvision.transforms as transforms # transformations (inc. random rotation)
from sklearn.model_selection import train_test_split 

In [37]:
class CNN(nn.Module):
    def __init__(self, in_channels = 4, num_classes = 1):
        super(CNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        
        # Max pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 12 * 12, 128) # 50 --> 25 --> 12
        self.fc2 = nn.Linear(128, 2)

        # Dropout layers
        self.dropout = nn.Dropout(0.3)  # Dropout with probability 0.3        

    def forward(self, x):
        # Convolutional layers with ReLU activation & Pooling layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        
        # Flatten
        x = x.flatten()

        # Fully connected layers with dropout
        x = self.fc1(x)
        x = self.dropout(x) 
        x = self.fc2(x)
        x = self.dropout(x) 
    
        # Sigmoid activation
        x = torch.sigmoid(x)

        return x

In [14]:
# Use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
# Hyperparameters (values?) 
in_channels = 4 
num_classes = 2
learning_rate = 0.001
batch_size = 32
num_epochs = 1

In [38]:
# Load data
features_class0 = np.load('test_area_non_CD_features.npy')
labels_class0 = np.load('test_area_non_CD_labels.npy')
features_class1 = np.load('test_area_CD_features.npy')
labels_class1 = np.load('test_area_CD_labels.npy')
# Remove aspect and edge
features_class0 = np.delete(features_class0, [2, 3, 6], axis=-1) 
features_class1 = np.delete(features_class1, [2, 3, 6], axis=-1)
# Take 80% for training (80% P, 80% N) and 20% for training (20% P, 20% N)
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(features_class0, labels_class0, test_size=0.2, random_state=42)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(features_class1, labels_class1, test_size=0.2, random_state=42)
# TO DO: Perform data augmentation on X_train_1 only
X_train_1_aug = X_train_1
y_train_1_aug = y_train_1
# Combine
X_train = np.vstack((X_train_0, X_train_1_aug))
y_train = np.hstack((y_train_0, y_train_1_aug)).astype(int)
X_test = np.vstack((X_test_0, X_test_1))
y_test = np.hstack((y_test_0, y_test_1)).astype(int)
# Convert to PyTorch tensors (size, channels, height, width)
X_train_tensor = torch.tensor(X_train).permute(0, 3, 1, 2)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test).permute(0, 3, 1, 2)
y_test_tensor = torch.tensor(y_test)
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# Print dataset sizes
print(f'Training set size: {len(train_dataset)}')
print(f'Test set size: {len(test_dataset)}')

Training set size: 8161
Test set size: 2041


In [39]:
# Functions (functions?)
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [40]:
# To DO: RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1179648 and 36864x128)
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1179648 and 36864x128)

In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader: 
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    model.train()

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)